In [128]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import re
import time
import csv
import numpy as np

In [129]:
main_url = 'https://www.iranketab.ir/book'

In [130]:
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36"}


In [131]:
""" response = requests.get(url=main_url , headers=headers)
print(response.status_code) """

' response = requests.get(url=main_url , headers=headers)\nprint(response.status_code) '

In [132]:
""" soup = BeautifulSoup(response.content, 'html.parser') """

" soup = BeautifulSoup(response.content, 'html.parser') "

In [133]:
""" page_number = 1  # Start with the first page

request_count = 0

session = requests.Session()
visited_urls = set()

with open('BooksUrls.csv', 'w', newline='') as csvfile:
    url_writer = csv.writer(csvfile)

    while True:
        current_page_url = f"{main_url}?pagenumber={page_number}&pagesize=20"
        response = session.get(current_page_url)

        if response.status_code != 200:
            break

        soup = BeautifulSoup(response.text, 'html.parser')
        allBooks = soup.find_all("a", class_="product-item-link")
        
        for book_url in allBooks:
            href = book_url.get("href")
            if href:
                full_url = f'{main_url}{href}'
                if full_url not in visited_urls:
                    visited_urls.add(full_url)
                    url_writer.writerow([full_url])
                    request_count += 1

                    if request_count % 500 == 0:
                        time.sleep(1)

        next_page_link = soup.find("a", {"data-page-no": str(page_number + 1)})

        if next_page_link:
            page_number += 1
        else:
            break """

' page_number = 1  # Start with the first page\n\nrequest_count = 0\n\nsession = requests.Session()\nvisited_urls = set()\n\nwith open(\'BooksUrls.csv\', \'w\', newline=\'\') as csvfile:\n    url_writer = csv.writer(csvfile)\n\n    while True:\n        current_page_url = f"{main_url}?pagenumber={page_number}&pagesize=20"\n        response = session.get(current_page_url)\n\n        if response.status_code != 200:\n            break\n\n        soup = BeautifulSoup(response.text, \'html.parser\')\n        allBooks = soup.find_all("a", class_="product-item-link")\n        \n        for book_url in allBooks:\n            href = book_url.get("href")\n            if href:\n                full_url = f\'{main_url}{href}\'\n                if full_url not in visited_urls:\n                    visited_urls.add(full_url)\n                    url_writer.writerow([full_url])\n                    request_count += 1\n\n                    if request_count % 500 == 0:\n                        time.sle

In [134]:
#csv_file = 'final_links.csv'

In [135]:
""" BookURLs = pd.read_csv('BooksUrls.csv', header=None)
urls = BookURLs[0].tolist()
urls = [url.replace("/book", "", 1) for url in urls]
link = pd.DataFrame({'final_books_Url': urls}) """

' BookURLs = pd.read_csv(\'BooksUrls.csv\', header=None)\nurls = BookURLs[0].tolist()\nurls = [url.replace("/book", "", 1) for url in urls]\nlink = pd.DataFrame({\'final_books_Url\': urls}) '

In [136]:
#link.to_csv('final_books_Url.csv', index=False)

In [137]:
BookURLs = pd.read_csv('final_books_Url.csv', header=None)
links = BookURLs[0].tolist()

In [148]:
csv_file = 'scraped_data.csv'

with open(csv_file, 'w', newline='', encoding='utf-8') as file:
    csv_writer = csv.DictWriter(file, fieldnames=[
        'book_id', 'print_series', 'size', 'type_of_print', 'translator_id', 'translator',
        'shabak', 'gregorian_publish_year', 'solar_publish_year', 'page_count',
        'Persian Title', 'English Title', 'Off', 'Rate', 'Break Price', 'Special_Price',
        'Exist', 'Publisher', 'publisher id', 'Writer', 'writer id', 'Description', 'Feature', 'Category'
    ])

    csv_writer.writeheader()

    session = requests.Session()
    request_count = 0

    for link in links[:10000]:
        response = session.get(link)
        if response.status_code == 200:
            soup = BeautifulSoup(response.content, 'html.parser')
            clearfix_elements = soup.find_all('div', class_='clearfix')
            writer_id = None
            publisher_id = None
            for clearfix in clearfix_elements:
                product_name_element = clearfix.find(class_='product-name')
                if product_name_element:
                    persian_title = product_name_element.text.strip()

                    en_name_element = clearfix.find('div', class_='product-name-englishname ltr')
                    if en_name_element:
                        english_title = en_name_element.text.strip()
                    else:
                        english_title = ""

                    off = clearfix.find('div', style='float: left;font-size: 12px;line-height: 1.375;background-color: #fb3449;color: #fff;padding: 5px 30px 3px;-webkit-border-radius: 0 16px 16px 16px;border-radius: 0 16px 16px 16px;')
                    if off:
                        off_percent = off.text.strip()
                    else:
                        off_percent = 0
                    rating_div = clearfix.find('div', class_='my-rating')
                    if rating_div:
                        data_rating = rating_div.get('data-rating')
                    else:
                        data_rating = 0
                    break_price = clearfix.find('span' , class_= 'price price-broken')
                    if break_price:
                        before_price = break_price.text.strip()
                    else :
                        before_price = np.nan
                    special_price = clearfix.find('span' , class_= 'price price-special')
                    if special_price:
                        after_price = special_price.text.strip()
                    else :
                        after_price = np.nan
                    exists_book_element = clearfix.find('li', class_='exists-book')
                    if exists_book_element:
                        exists_book_text = exists_book_element.text.strip()
                    else:
                        exists_book_text = 'ناموجود'
                    publisher_element = clearfix.find('div', class_='col-xs-12 prodoct-attribute-items')
                    if publisher_element:
                        publisher_span = publisher_element.find('span', class_='prodoct-attribute-item')
                        if publisher_span and publisher_span.text.strip() == 'انتشارات:':
                            publisher_text = publisher_element.find('span', class_='prodoct-attribute-item').find_next('span').text.strip()
                        else:
                            publisher_text = ""
                    else:
                        publisher_text = ""
                    #########################################################################################################################################
                    look_for_id = clearfix.find('div', class_='row clearfix')
                    for id_publisher_and_writer in look_for_id.find_all('a'):
                        if id_publisher_and_writer:
                            href = id_publisher_and_writer.get('href')
                            if 'publisher' in href:
                                publisher_id = int(href.split('/publisher/')[1].split('-')[0])
                            elif 'profile' in href:
                                writer_id = int(href.split('/profile/')[1].split('-')[0])
                    #########################################################################################################################################
                    writer_element = clearfix.find('span', itemprop='name')
                    if writer_element:
                        writer_text = writer_element.text.strip()
                    else:
                        writer_text = ""

                    description_element = soup.find('div', class_='product-description')
                    if description_element:
                        description_text = description_element.text.strip()
                    else:
                        description_text = ""
                    feature_elements = soup.find_all('div', class_='product-features')
                    features = []
                    for feature_element in feature_elements:
                        h4_elements = feature_element.find_all('h4')
                        for h4_element in h4_elements:
                            feature_text = h4_element.text.strip()
                            features.append(feature_text)
                    product_tags_div = soup.find('div', class_='product-tags')
                    if product_tags_div:
                        h5_elements = product_tags_div.find_all('h5')
                        tags = [h5.text.strip() for h5 in h5_elements]
                    else:
                        tags = []
                    book_id = None
                    translator = []
                    shabak = None
                    page_count = None
                    solar_publish_year = None
                    gregorian_publish_year = None
                    print_series = None
                    type_of_print = None
                    size = None
                    product_info = clearfix.find('table', class_='product-table')
                    if product_info:
                        attr_elements = product_info.find_all('td')
                        for i in range(0, len(attr_elements), 2):
                            attr_name = attr_elements[i].text.strip()
                            attr_value = attr_elements[i + 1].text.strip()
                            if attr_name == 'کد کتاب :':
                                book_id = attr_value
                            if 'مترجم' in attr_name:
                                translator_element = attr_elements[i + 1].find('a', itemprop='author')
                                if translator_element:
                                    translator_id = int(translator_element.get('href').split('/profile/')[1].split('-')[0])
                                    translator.append(attr_value)
                                else:
                                    translator_id = None
                            if 'شابک' in attr_name:
                                shabak = attr_value
                            if 'تعداد صفحه' in attr_name:
                                page_count = attr_value
                            if 'سال انتشار شمسی' in attr_name:
                                solar_publish_year = attr_value
                            if 'سال انتشار میلادی' in attr_name:
                                gregorian_publish_year = attr_value
                            if 'نوع جلد' in attr_name:
                                type_of_print = attr_value
                            if 'سری چاپ' in attr_name:
                                print_series = attr_value
                            if 'قطع' in attr_name:
                                size = attr_value
                    scraped_data = {
                        'book_id': book_id,
                        'print_series': print_series,
                        'size': size,
                        'type_of_print': type_of_print,
                        'translator_id': translator_id,
                        'translator': translator,
                        'shabak': shabak,
                        'gregorian_publish_year': gregorian_publish_year,
                        'solar_publish_year': solar_publish_year,
                        'page_count': page_count,
                        'Persian Title': persian_title,
                        'English Title': english_title,
                        'Off': off_percent,
                        'Rate': round(float(data_rating), 2),
                        'Break Price': before_price,
                        'Special_Price': after_price,
                        'Exist': exists_book_text,
                        'Publisher': publisher_text,
                        'publisher id': publisher_id,
                        'Writer': writer_text,
                        'writer id': writer_id,
                        'Description': description_text,
                        'Feature': features,
                        'Category': tags
                    }
                    csv_writer.writerow(scraped_data)
                    
                request_count += 1
                if request_count % 500 == 0:
                    time.sleep(1)

In [143]:
book = pd.DataFrame(scraped_data)
book.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 24 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   book_id                 1 non-null      object 
 1   print_series            1 non-null      object 
 2   size                    1 non-null      object 
 3   type_of_print           1 non-null      object 
 4   translator_id           1 non-null      int64  
 5   translator              1 non-null      object 
 6   shabak                  1 non-null      object 
 7   gregorian_publish_year  1 non-null      object 
 8   solar_publish_year      1 non-null      object 
 9   page_count              1 non-null      object 
 10  Persian Title           1 non-null      object 
 11  English Title           1 non-null      object 
 12  Off                     1 non-null      object 
 13  Rate                    1 non-null      float64
 14  Break Price             1 non-null      object

In [144]:
book.drop_duplicates(subset=['Persian Title', 'English Title' , 'Off'	,'Rate'	,'Break Price',	'Special_Price'	,'Exist',	'Publisher'	,'Writer'], inplace=True)

In [145]:
book.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 24 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   book_id                 1 non-null      object 
 1   print_series            1 non-null      object 
 2   size                    1 non-null      object 
 3   type_of_print           1 non-null      object 
 4   translator_id           1 non-null      int64  
 5   translator              1 non-null      object 
 6   shabak                  1 non-null      object 
 7   gregorian_publish_year  1 non-null      object 
 8   solar_publish_year      1 non-null      object 
 9   page_count              1 non-null      object 
 10  Persian Title           1 non-null      object 
 11  English Title           1 non-null      object 
 12  Off                     1 non-null      object 
 13  Rate                    1 non-null      float64
 14  Break Price             1 non-null      object

In [146]:
book.head()

,book_id,print_series,size,type_of_print,translator_id,translator,shabak,gregorian_publish_year,solar_publish_year,page_count,...,Break Price,Special_Price,Exist,Publisher,publisher id,Writer,writer id,Description,Feature,Category
0,43,80,رقعی,جلد سخت,164,[پیمان خاکسار],978-600229-5002,2008,1402,656,...,"395,000","375,250",موجود,نشر چشمه,33,استیو تولتز,27,جسپر دین، بیشتر عمرش نمی توانست تصمیم بگیرد چه...,[برنده جایزه ی NSW Premier سال 2009],"[جایزه ی ان اس دبلیو پرایمر, ادبیات استرالیا, ..."


In [ ]:
book.to_csv('info_book.csv', index=False)

## CLEANING DATA